**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Istio Canary

## Hello Istio

Very basic traffic management
https://istio.io/blog/2018/incremental-traffic-management/



In [1]:
kubectl label namespace default istio-injection=enabled

namespace/default labeled


## Create a test pod

In [2]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: sleep
  labels:
    app: sleep
spec:
  ports:
  - port: 80
    name: http
  selector:
    app: sleep
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
---
EOF

service/sleep created
deployment.apps/sleep created


In [1]:
export SLEEP_POD=$(kubectl get pod -l app=sleep \
  -o jsonpath={.items..metadata.name})
echo $SLEEP_POD

sleep-56bc8d768-fmxxt


In [4]:
kubectl get destinationrule 

No resources found in default namespace.


In [5]:
kubectl get virtualservice

No resources found in default namespace.


In [6]:
kubectl get deployments

NAME    READY   UP-TO-DATE   AVAILABLE   AGE
sleep   1/1     1            1           12s


In [7]:
kubectl get pods

NAME                    READY   STATUS    RESTARTS   AGE
sleep-56bc8d768-fmxxt   2/2     Running   0          18s


In [8]:
kubectl get pod $SLEEP_POD -o yaml | grep --color=always  -E '$|.*image:.*'

apiVersion: v1
kind: Pod
metadata:
  annotations:
    prometheus.io/path: /stats/prometheus
    prometheus.io/port: "15020"
    prometheus.io/scrape: "true"
    sidecar.istio.io/status: '{"version":"e2cb9d4837cda9584fd272bfa1f348525bcaacfadb7e9b9efbd21a3bb44ad7a1","initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-ca-cert"],"imagePullSecrets":null}'
  creationTimestamp: "2021-01-27T13:38:58Z"
  generateName: sleep-56bc8d768-
  labels:
    app: sleep
    istio.io/rev: default
    pod-template-hash: 56bc8d768
    security.istio.io/tlsMode: istio
    service.istio.io/canonical-name: sleep
    service.istio.io/canonical-revision: latest
  managedFields:
  - apiVersion: v1
    fieldsType: FieldsV1
    fieldsV1:
      f:metadata:
        f:generateName: {}
        f:labels:
          .: {}
          f:app: {}
          f:pod-template-hash: {}
        f:ownerReferences:
          .: {}
          k:{"uid":"da

# Admisssion Controlers and Sidecars

- Istio works as an [ Admission Controller](https://kubernetes.io/blog/2019/03/21/a-guide-to-kubernetes-admission-controllers/)
- adding a [Sidecar Pattern](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45406.pdf)
- [Kubernetes Patterns](https://www.redhat.com/cms/managed-files/cm-oreilly-kubernetes-patterns-ebook-f19824-201910-en.pdf)
- [Envoy Proxy](https://www.envoyproxy.io/)
- [Patterns with Envoy Proxy](https://blog.christianposta.com/microservices/00-microservices-patterns-with-envoy-proxy-series/)

In [3]:
kubectl exec $SLEEP_POD -- curl -L heise.de | wc -l

Defaulting container name to sleep.
Use 'kubectl describe pod/sleep-56bc8d768-fmxxt -n default' to see all of the containers in this pod.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   229  100   229    0     0   3367      0 --:--:-- --:--:-- --:--:--  3318
100  520k  100  520k    0     0  1319k      0 --:--:-- --:--:-- --:--:-- 1319k
28176


# Istio Sample Application

In [5]:
cd istio-*

In [6]:
pwd

/minikube-host/training/notebooks/istio-1.8.2


In [7]:
kubectl apply -f samples/bookinfo/platform/kube/bookinfo.yaml

service/details created
serviceaccount/bookinfo-details created
deployment.apps/details-v1 created
service/ratings created
serviceaccount/bookinfo-ratings created
deployment.apps/ratings-v1 created
service/reviews created
serviceaccount/bookinfo-reviews created
deployment.apps/reviews-v1 created
deployment.apps/reviews-v2 created
deployment.apps/reviews-v3 created
service/productpage created
serviceaccount/bookinfo-productpage created
deployment.apps/productpage-v1 created


In [12]:
kubectl get pods 

NAME                              READY   STATUS    RESTARTS   AGE
details-v1-79c697d759-rkjvf       2/2     Running   0          2m34s
productpage-v1-65576bb7bf-d4gt5   2/2     Running   0          2m33s
ratings-v1-7d99676f7f-2ncvj       2/2     Running   0          2m34s
reviews-v1-987d495c-j5qs6         2/2     Running   0          2m33s
reviews-v2-6c5bf657cf-gsr79       2/2     Running   0          2m33s
reviews-v3-5f7b9f4f77-n5j8f       2/2     Running   0          2m33s
sleep-56bc8d768-fmxxt             2/2     Running   0          14m


In [13]:
kubectl get services

NAME          TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
details       ClusterIP   10.99.240.114   <none>        9080/TCP   2m42s
kubernetes    ClusterIP   10.96.0.1       <none>        443/TCP    20h
productpage   ClusterIP   10.111.229.39   <none>        9080/TCP   2m41s
ratings       ClusterIP   10.100.74.5     <none>        9080/TCP   2m42s
reviews       ClusterIP   10.108.2.171    <none>        9080/TCP   2m42s
sleep         ClusterIP   10.101.36.20    <none>        80/TCP     14m


In [14]:
kubectl exec "$(kubectl get pod -l app=ratings -o jsonpath='{.items[0].metadata.name}')" \
  -c ratings -- curl -s productpage:9080/productpage | grep --color=always -E "$|<title>.*</title>"

<!DOCTYPE html>
<html>
  <head>
    <title>Simple Bookstore App</title>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1.0">

<!-- Latest compiled and minified CSS -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap.min.css">

<!-- Optional theme -->
<link rel="stylesheet" href="static/bootstrap/css/bootstrap-theme.min.css">

  </head>
  <body>
    
    

<nav class="navbar navbar-inverse navbar-static-top">
  <div class="container">
    <div class="navbar-header">
      <a class="navbar-brand" href="#">BookInfo Sample</a>
    </div>
    
    <button type="button" class="btn btn-default navbar-btn navbar-right" data-toggle="modal" href="#login-modal">Sign
      in</button>
    
  </div>
</nav>

<!---
<div class="navbar navbar-inverse navbar-fixed-top">
  <div class="container">
    <div class="navbar-header pull-left">
      <a class="navbar-brand" href="#">Microservices Fabric Bo

In [15]:
cat samples/bookinfo/networking/bookinfo-gateway.yaml

apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: bookinfo-gateway
spec:
  selector:
    istio: ingressgateway # use istio default controller
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "*"
---
apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: bookinfo
spec:
  hosts:
  - "*"
  gateways:
  - bookinfo-gateway
  http:
  - match:
    - uri:
        exact: /productpage
    - uri:
        prefix: /static
    - uri:
        exact: /login
    - uri:
        exact: /logout
    - uri:
        prefix: /api/v1/products
    route:
    - destination:
        host: productpage
        port:
          number: 9080


In [16]:
kubectl apply -f samples/bookinfo/networking/bookinfo-gateway.yaml

gateway.networking.istio.io/bookinfo-gateway created
virtualservice.networking.istio.io/bookinfo created


In [17]:
export PATH=$PATH:$(pwd)/bin

In [18]:
istioctl analyze


✔ No validation issues found when analyzing namespace: default.


In [19]:
export INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="http2")].nodePort}')
export SECURE_INGRESS_PORT=$(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.spec.ports[?(@.name=="https")].nodePort}')
export INGRESS_HOST=$(hostname -i)
echo INGRESS_PORT=$INGRESS_PORT
echo SECURE_INGRESS_PORT=$SECURE_INGRESS_PORT
echo INGRESS_HOST=$INGRESS_HOST

INGRESS_PORT=31888
SECURE_INGRESS_PORT=30314
INGRESS_HOST=192.168.49.2


In [20]:
export GATEWAY_URL=$INGRESS_HOST:$INGRESS_PORT
echo "http://$GATEWAY_URL/productpage"

http://192.168.49.2:31888/productpage


In [22]:
kubectl apply -f samples/addons

serviceaccount/grafana unchanged
configmap/grafana unchanged
service/grafana unchanged
deployment.apps/grafana configured
configmap/istio-grafana-dashboards configured
configmap/istio-services-grafana-dashboards configured
deployment.apps/jaeger unchanged
service/tracing unchanged
service/zipkin unchanged
service/jaeger-collector unchanged
customresourcedefinition.apiextensions.k8s.io/monitoringdashboards.monitoring.kiali.io unchanged
serviceaccount/kiali unchanged
configmap/kiali unchanged
clusterrole.rbac.authorization.k8s.io/kiali-viewer unchanged
clusterrole.rbac.authorization.k8s.io/kiali unchanged
clusterrolebinding.rbac.authorization.k8s.io/kiali unchanged
service/kiali unchanged
deployment.apps/kiali unchanged
monitoringdashboard.monitoring.kiali.io/envoy created
monitoringdashboard.monitoring.kiali.io/go created
monitoringdashboard.monitoring.kiali.io/kiali created
monitoringdashboard.monitoring.kiali.io/micrometer-1.0.6-jvm-pool created
monitoringdashboard.monitoring.kiali.io

In [24]:
istioctl dashboard kiali --address $(hostname -i) | sed s/localhost/$(hostname -i)/

http://192.168.49.2:20001/kiali
Failed to open browser; open http://192.168.49.2:20001/kiali in your browser.



## Cleanup

In [26]:
kubectl delete -f samples/addons -f samples/bookinfo/platform/kube/bookinfo.yaml

Error from server (NotFound): error when deleting "samples/addons/grafana.yaml": serviceaccounts "grafana" not found
Error from server (NotFound): error when deleting "samples/addons/grafana.yaml": configmaps "grafana" not found
Error from server (NotFound): error when deleting "samples/addons/grafana.yaml": services "grafana" not found
Error from server (NotFound): error when deleting "samples/addons/grafana.yaml": deployments.apps "grafana" not found
Error from server (NotFound): error when deleting "samples/addons/grafana.yaml": configmaps "istio-grafana-dashboards" not found
Error from server (NotFound): error when deleting "samples/addons/grafana.yaml": configmaps "istio-services-grafana-dashboards" not found
Error from server (NotFound): error when deleting "samples/addons/jaeger.yaml": deployments.apps "jaeger" not found
Error from server (NotFound): error when deleting "samples/addons/jaeger.yaml": services "tracing" not found
Error from server (NotFound): error when deleting "

: 1